In [1]:
pip install pypdf

   ---------------------------------------- 0.0/295.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/295.8 kB ? eta -:--:--
   --------------------------------- ------ 245.8/295.8 kB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 295.8/295.8 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
pip install pycryptodome


In [32]:

from pypdf import PdfReader

# PDF 파일 열기
with open('/content/drive/MyDrive/Colab Notebooks/실전_프로젝트/데이터/제출서류 1. 신청용 사업계획서 -.pdf', 'rb') as file:
    reader = PdfReader(file)
    # 총 페이지 수 확인
    num_pages = len(reader.pages)

    # 모든 페이지 텍스트 추출
    all_text = ""
    for page_num in range(num_pages):
        page = reader.pages[page_num]
        text = page.extract_text()

        # 인코딩 오류 무시 및 텍스트 누적
        corrected_text = text.encode('utf-8', errors='ignore').decode('utf-8')
        all_text += corrected_text + "\n"  # 페이지 간 텍스트 구분을 위해 줄 바꿈 추가

# 추출한 텍스트를 txt 파일로 저장
output_file_path = '/content/drive/MyDrive/Colab Notebooks/실전_프로젝트/데이터/텍스트_데이터/제출서류 1. 신청용 사업계획서 -.txt'
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    output_file.write(all_text)

print(f"텍스트가 성공적으로 {output_file_path}에 저장되었습니다.")


텍스트가 성공적으로 /content/drive/MyDrive/Colab Notebooks/실전_프로젝트/데이터/텍스트_데이터/제출서류 1. 신청용 사업계획서 -.txt에 저장되었습니다.


In [48]:
!hwp5txt

usage: hwp5txt [-h] [--version] [--loglevel LOGLEVEL] [--logfile LOGFILE] [--output OUTPUT]
               <hwp5file>
hwp5txt: error: the following arguments are required: <hwp5file>


In [49]:
import olefile
import zlib
import struct

def get_hwp_text(filename):
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()

    # HWP 파일 검증
    if ["FileHeader"] not in dirs or \
       ["\x05HwpSummaryInformation"] not in dirs:
        raise Exception("Not Valid HWP.")

    # 문서 포맷 압축 여부 확인
    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    # Body Sections 불러오기
    nums = []
    for d in dirs:
        if d[0] == "BodyText":
            nums.append(int(d[1][len("Section"):]))
    sections = ["BodyText/Section"+str(x) for x in sorted(nums)]

    # 전체 text 추출
    text = ""
    for section in sections:
        bodytext = f.openstream(section)
        data = bodytext.read()
        if is_compressed:
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data

        # 각 Section 내 text 추출
        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in [67]:
                rec_data = unpacked_data[i+4:i+4+rec_len]
                section_text += rec_data.decode('utf-16')
                section_text += "\n"

            i += 4 + rec_len

        text += section_text
        text += "\n"

    return text